In [17]:
import tensorflow as tf
import datetime, os
from model import *
from data import *

#### Unpack videos and labels

In [18]:
unpack_video('data/larynx/train', video_folder='video', image_folder='image', target_size=(256, 256))
unpack_tif('data/larynx/train', tif_folder='tifs', label_folder='label', target_size=(256, 256))

#### Launch TensorBoard Session

In [20]:
%load_ext tensorboard
%reload_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


#### Initialize the model (U-Net)

In [21]:
model = unet()
model_checkpoint = ModelCheckpoint('unet_larynx.hdf5', monitor='loss', verbose=1, save_best_only=True)

#### Prepare training dataset

In [22]:
training_dataset = training_dataset_generator(2,
                                              'data/larynx/train',
                                              'image',
                                              'label',
                                              dict([]),
                                              save_to_dir=None)

In [23]:
# Set up TensorBoard callback
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Early stopping callback
early_stopping = EarlyStopping(monitor='loss', patience=7, mode='auto', restore_best_weights=True)


# Train the model using the training dataset and TensorBoard callback
model.fit(training_dataset, 
          steps_per_epoch=10, 
          epochs=200, 
          callbacks=[model_checkpoint, tensorboard_callback])

NameError: name 'EarlyStopping' is not defined

#### Testing the model

In [16]:
# testing the model
test_dataset = test_dataset_generator("data/larynx/test", num_image=5)
results = model.predict(test_dataset, 3, verbose=1)
saveResult("data/larynx/test", results)


5/5 [==============================] - 1s 164ms/step
